# 비슷한 노래 장르를 담은 playlist를 추천하자

In [1]:
# 모듈 불러오기
import os
import json
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import scipy.sparse as spr
from tqdm.notebook import tqdm

from itertools import chain

In [2]:
# 데이터 불러오기
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)

genre_gn_all = pd.read_json('data/genre_gn_all.json', typ = 'series') # 장르 읽어오기

train = pd.DataFrame.from_dict(train_dict)
song = pd.DataFrame.from_dict(song_dict)
# 장르코드 : gnr_code, 장르명 : gnr_name
genre = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})    



In [3]:
# data 살펴보기
display(train)
display(song)
display(genre)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


,gnr_code,gnr_name
0,GN0100,발라드
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00
...,...,...
249,GN2900,뮤지컬
250,GN2901,세부장르전체
251,GN2902,국내뮤지컬
252,GN2903,국외뮤지컬


# 원본데이터로 해보자

In [28]:
# train copy
df_train = train.copy()
display(df_train.head(1))

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000


---
실험

In [29]:
# # song copy
# # song data 사용할 컬럼만 가져오기
# df_song = song[['song_gn_dtl_gnr_basket','id']].copy()
# # 컬럼명 한글로 바꾸기
# df_song = df_song.rename(columns={'song_gn_dtl_gnr_basket':'장르','id':'곡id'})

## train DF에 [장르, 장르id] columns를 만들어주자

In [30]:
from itertools import chain
genre_code_list = list(genre_dict.keys())
genre_id_dict = dict(zip(genre_code_list,range(len(genre_code_list))))
# id_genre_dict = dict(zip(range(len(genre_code)),genre_code))

In [31]:
# 모든 노래에 대한 장르 딕셔너리 만들기 # dict(song_id : gnr_id) 
songid_genre_dict = dict(zip(df_song['곡id'].tolist(),df_song['장르'].tolist()))
# songid_genre_dict # dict(모든 노래id : 장르)

In [44]:
# df_train data에 장르 column 만들어주기
from itertools import chain

song_id_lists = df_train['songs'].tolist() # 이중 리스트
song_genre_lists = [] # 빈 리스트

for song_id_list in song_id_lists:  # song_id_lists <- song_id_list <- song_id 
    temp = [] # 빈 리스트
    for song_id in song_id_list: # song_id_list <- song_id <- temp
        temp.append(songid_genre_dict[song_id]) 
    song_genre_lists.append(list(chain.from_iterable(temp))) # 이중리스트를 플랫하게 만듦.
    
df_train['장르'] = song_genre_lists
df_train.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date,장르,updt_year,updt_year_month
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19,"[GN1402, GN1401, GN0901, GN0902, GN1001, GN101...",2013,2013 12


In [46]:
song_genre_lists

[['GN1402',
  'GN1401',
  'GN0901',
  'GN0902',
  'GN1001',
  'GN1012',
  'GN1005',
  'GN1001',
  'GN1013',
  'GN0901',
  'GN0902',
  'GN1001',
  'GN1013',
  'GN0901',
  'GN0902',
  'GN1001',
  'GN1013',
  'GN0901',
  'GN0902',
  'GN1001',
  'GN1003',
  'GN1013',
  'GN1001',
  'GN0901',
  'GN1304',
  'GN1301',
  'GN1302',
  'GN0904',
  'GN1402',
  'GN1401',
  'GN0901',
  'GN1001',
  'GN1001',
  'GN1912',
  'GN1904',
  'GN1901',
  'GN0901',
  'GN0901',
  'GN0902',
  'GN1001',
  'GN1402',
  'GN1401',
  'GN0901',
  'GN1003',
  'GN1001',
  'GN0901',
  'GN1102',
  'GN1101'],
 ['GN0101',
  'GN0103',
  'GN0601',
  'GN0605',
  'GN0104',
  'GN0101',
  'GN0601',
  'GN0605',
  'GN0104',
  'GN0101',
  'GN0601',
  'GN0605',
  'GN0104',
  'GN0101',
  'GN0508',
  'GN0501',
  'GN0601',
  'GN0503',
  'GN0605',
  'GN0508',
  'GN0501',
  'GN0601',
  'GN0503',
  'GN0605',
  'GN0805',
  'GN0501',
  'GN0502',
  'GN0801',
  'GN0509',
  'GN0805',
  'GN0501',
  'GN0502',
  'GN0801',
  'GN0509',
  'GN0105',
  '

In [33]:
# train 데이터에 genre를 붙인 csv파일 만들기
# df_train.to_csv('data/train_genre.csv') 

## updt_year / month로 나눠주기
> 19년도 5월-20년도 4월까지의 1년 치 df만 뽑아오기     
> 각 월 별 장르 리스트를 groupby로 합쳐버리기     
> 장르 : cnt개수의 딕셔너리 만들기     

In [34]:
# updt_date시리즈를 datetime 형식으로 바꾸기 
df_train['updt_date'] = pd.to_datetime(df_train['updt_date'])

In [35]:
df_train['updt_year'] = df_train['updt_date'].dt.year

In [36]:
df_train['updt_year_month'] = df_train['updt_date'].dt.strftime('%Y %m')
# df_train['updt_month'] = df_train['updt_date'].dt.strftime('%m')
df_train.head(2)

,tags,id,plylst_title,songs,like_cnt,updt_date,장르,updt_year,updt_year_month
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19,"[GN1402, GN1401, GN0901, GN0902, GN1001, GN101...",2013,2013 12
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42,"[GN0101, GN0103, GN0601, GN0605, GN0104, GN010...",2014,2014 12


In [37]:
# 19년도와 20년도 4월까지의 1년 치 df만 뽑아오기 
df_train_19 = df_train[df_train['updt_year_month'].str[:4] == '2019']
df_train_19.info()
df_train_20 = df_train[df_train['updt_year_month'].str[:4] == '2020']
df_train_20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21289 entries, 3 to 115069
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   tags             21289 non-null  object        
 1   id               21289 non-null  int64         
 2   plylst_title     21289 non-null  object        
 3   songs            21289 non-null  object        
 4   like_cnt         21289 non-null  int64         
 5   updt_date        21289 non-null  datetime64[ns]
 6   장르               21289 non-null  object        
 7   updt_year        21289 non-null  int64         
 8   updt_year_month  21289 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16607 entries, 10 to 115070
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   tags             1

In [38]:
# 19년도 곡과 20년도 곡 합치기 
# -> 이건 나중에 할 것. 19년도 1-4월과 20년도 1-4월이 겹침.
# df_train_19_20 = pd.concat([df_train_19, df_train_20],axis=0)
# df_train_19_20.info()

In [158]:
# 사용할 컬럼만 가져오자
df_train_20_ = df_train_20[['id','plylst_title','장르','updt_year_month']]

In [159]:
df_train_20_

,id,plylst_title,장르,updt_year_month
10,31804,걸그룹 땐쓰쏭,"[GN2503, GN0204, GN2501, GN2506, GN0201, GN250...",2020 04
21,12460,ᴡʜɪᴛᴇ : ʀᴏᴍᴀɴᴛɪᴄ ᴊᴀᴢᴢ ᴀᴛ ᴀ ᴄᴀғᴇ,"[GN1701, GN1701, GN1703, GN1702, GN1701, GN170...",2020 03
27,144318,사랑스러운 남녀듀엣,"[GN0105, GN0101, GN0105, GN0101, GN0105, GN150...",2020 03
30,24779,"거슈윈에서 스타워즈까지, 미국 대표 클래식","[GN1601, GN1604, GN1601, GN1605, GN1601, GN160...",2020 02
34,53737,믿고듣는 하우스 일렉트로니카 POP #2,"[GN1104, GN1103, GN1101, GN1104, GN2704, GN110...",2020 02
...,...,...,...,...
115024,113924,[매주 업데이트] 슈가맨 시즌3 슈가송 모음,"[GN0104, GN0101, GN0501, GN0601, GN0503, GN060...",2020 03
115027,2410,따뜻한 봄을 알리는 음악,"[GN1806, GN1803, GN1801, GN1803, GN1801, GN150...",2020 02
115045,23046,2020년 1월,"[GN0509, GN0601, GN0503, GN0606, GN0501, GN080...",2020 01
115066,120325,METAL E'SM #2,"[GN1006, GN1013, GN1001, GN1007, GN1013, GN100...",2020 04


In [160]:
from collections import defaultdict,Counter

genre_by_month = defaultdict(list) # 빈 { [ ] }

for genre,month in zip(df_train_20_['장르'].tolist(),df_train_20_['updt_year_month'].tolist()):  # { month(key) : genre(value) }
    genre_by_month[month].extend(genre)
    
    
    
genre_by_month_count = {}

for k,v in genre_by_month.items():
    genre_by_month_count[k] = dict(Counter(v))
    
df_train_20_['장르_카운트'] = df_train_20_['updt_year_month'].map(lambda x : genre_by_month_count[x])

df_train_20_.head()

# defaultdict, Counter, extend, for i in zip( ), for i in range

<ipython-input-160-d8780e56769c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_20_['장르_카운트'] = df_train_20_['updt_year_month'].map(lambda x : genre_by_month_count[x])


,id,plylst_title,장르,updt_year_month,장르_카운트
10,31804,걸그룹 땐쓰쏭,"[GN2503, GN0204, GN2501, GN2506, GN0201, GN250...",2020 04,"{'GN2503': 25023, 'GN0204': 5089, 'GN2501': 47..."
21,12460,ᴡʜɪᴛᴇ : ʀᴏᴍᴀɴᴛɪᴄ ᴊᴀᴢᴢ ᴀᴛ ᴀ ᴄᴀғᴇ,"[GN1701, GN1701, GN1703, GN1702, GN1701, GN170...",2020 03,"{'GN1701': 5507, 'GN1703': 1045, 'GN1702': 142..."
27,144318,사랑스러운 남녀듀엣,"[GN0105, GN0101, GN0105, GN0101, GN0105, GN150...",2020 03,"{'GN1701': 5507, 'GN1703': 1045, 'GN1702': 142..."
30,24779,"거슈윈에서 스타워즈까지, 미국 대표 클래식","[GN1601, GN1604, GN1601, GN1605, GN1601, GN160...",2020 02,"{'GN1601': 2954, 'GN1604': 218, 'GN1605': 298,..."
34,53737,믿고듣는 하우스 일렉트로니카 POP #2,"[GN1104, GN1103, GN1101, GN1104, GN2704, GN110...",2020 02,"{'GN1601': 2954, 'GN1604': 218, 'GN1605': 298,..."


In [161]:
genre_by_month_count_df=pd.DataFrame.from_dict(genre_by_month_count,orient='index') # orient

In [162]:
# 2020년도 분기 
genre_by_month_count_df = genre_by_month_count_df.fillna(0)
# genre_by_month_count_df.to_csv('data/genre_by_month_count_df.csv')

In [163]:
df_train_20_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16607 entries, 10 to 115070
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               16607 non-null  int64 
 1   plylst_title     16607 non-null  object
 2   장르               16607 non-null  object
 3   updt_year_month  16607 non-null  object
 4   장르_카운트           16607 non-null  object
dtypes: int64(1), object(4)
memory usage: 778.5+ KB


In [164]:
genre_by_month_count_df

,GN2503,GN0204,GN2501,GN2506,GN0201,GN0205,GN0303,GN2504,GN0301,GN0401,...,GN2215,GN2301,GN1611,GN2204,GN1609,GN2206,GN2302,GN2209,GN2406,GN2213
2020 04,25023,5089,47822,27051,36857,24468,19781,6071,40914,33023,...,9,70.0,12,8.0,2,0.0,0.0,0.0,0.0,0.0
2020 03,14818,3120,30443,15407,21620,13575,12870,4628,24750,19266,...,40,10.0,17,65.0,1,71.0,2.0,2.0,2.0,0.0
2020 02,9353,1690,17811,9217,13099,8640,8337,2713,16076,12820,...,318,198.0,18,441.0,1,157.0,15.0,63.0,1.0,19.0
2020 01,7970,1710,14746,8536,11854,7545,6343,1502,12201,10576,...,1,0.0,6,0.0,1,0.0,0.0,0.0,1.0,0.0


In [165]:
# 각 월 별 장르 리스트를 groupby로 합쳐버리기
df_train_20_.groupby(df_train_20_['updt_year_month'], axis=1)

In [166]:
df_train_19 = df_train[df_train['updt_year_month'].str[:4] == '2019']

In [23]:
# df_train dataframe에 장르id column 만들어주기
df_train['장르id'] = df_train['장르'].map(lambda x : [ genre_id_dict[i] for i in x])
df_train

,tags,id,plylst_title,songs,like_cnt,updt_date,장르,장르id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[GN1402, GN1401, GN0901, GN0902, GN1001, GN101...","[111, 110, 57, 58, 66, 77, 70, 66, 78, 57, 58,..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[GN0101, GN0103, GN0601, GN0605, GN0104, GN010...","[1, 3, 33, 37, 4, 1, 33, 37, 4, 1, 33, 37, 4, ..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[GN0401, GN0403, GN0401, GN0403, GN0501, GN060...","[19, 21, 19, 21, 23, 33, 25, 38, 31, 19, 21, 1..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[GN0908, GN1509, GN0901, GN2207, GN1501, GN150...","[64, 126, 57, 205, 118, 123, 1, 3, 63, 57, 101..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,"[GN0101, GN0101, GN0103, GN2502, GN2506, GN250...","[1, 1, 3, 227, 231, 226, 9, 7, 9, 7, 228, 10, ..."
...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,"[GN1006, GN1013, GN1001, GN1007, GN1013, GN100...","[71, 78, 66, 72, 78, 66, 72, 78, 66, 72, 78, 6..."
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,"[GN1104, GN1101, GN1104, GN1102, GN1101, GN110...","[84, 81, 84, 82, 81, 84, 83, 81, 84, 83, 81, 8..."
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,"[GN0105, GN0101, GN0105, GN0101, GN2502, GN060...","[5, 1, 5, 1, 227, 33, 226, 37, 1, 19, 21, 4, 1..."
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,"[GN1107, GN1102, GN1101, GN1013, GN1008, GN100...","[87, 82, 81, 78, 73, 66, 66, 120, 118, 73, 68,..."


# sparse matrix로 만들자

In [24]:
from collections import defaultdict, Counter

my_dict = defaultdict(dict)

genre_lists = df_train['장르id'].tolist()
# genre_lists

In [25]:
for i in range(len(df_train)):
    my_dict[i] = dict(Counter(genre_lists[i]))
row =[]
col =[]
dat =[]

for k, v in my_dict.items():
    for vk,vv in v.items():
        row.append(k)
        col.append(vk)
        dat.append(vv)

print(row)
print(col)
print(dat)

# row ply, col genre, dat 
df_spr_plylst = spr.csr_matrix((dat, (row, col)), shape=(len(df_train), len(genre)))
df_spr_plylst

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



<115071x254 sparse matrix of type '<class 'numpy.intc'>'
	with 2081861 stored elements in Compressed Sparse Row format>

In [26]:
df_spr_plylst.todense()


matrix([[ 0,  0,  0, ...,  0,  0,  0],
        [ 0, 11,  0, ...,  0,  0,  0],
        [ 0,  2,  0, ...,  0,  0,  0],
        ...,
        [ 0,  8,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)